# SET UP ENVIRONMET

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'durable-utility-367514'

In [2]:
REGION = 'europe-west4'


In [3]:
from google.cloud import storage

import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'google'

In [4]:
gcs = storage.Client(project = PROJECT_ID)

NameError: name 'storage' is not defined

In [5]:
BUCKET = PROJECT_ID

---
## Create Storage Bucket
Check to see if bucket already exist and create if missing:
- [GCS Python Client](https://cloud.google.com/python/docs/reference/storage/latest/google.cloud.storage.client.Client)

In [6]:
if not gcs.lookup_bucket(BUCKET):
    bucketDef = gcs.bucket(BUCKET)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(f'Created Bucket: {gcs.lookup_bucket(BUCKET).name}')
else:
    bucketDef = gcs.bucket(BUCKET)
    print(f'Bucket already exist: {bucketDef.name}')

NameError: name 'gcs' is not defined

---
<a id = 'permissions'></a>
## Service Account & Permissions

This notebook instance is running as a service account in GCP.  This service account will also be used to run other services in Vertex AI like training jobs and pipelines.  The service account will need permission to interact with object in Cloud Storage which requires the role ([roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles)).  

In [7]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

"ERROR: (gcloud.config.list) Name expected [config *HERE* 'value(core.account)']."

In [8]:
!gcloud services enable cloudresourcemanager.googleapis.com

ERROR: (gcloud.services.enable) PERMISSION_DENIED: Project 'durable-utility-367514' not found or permission denied.
Help Token: AZWD64r8R_1cORJ-sZ4wRw3ywOUnrZcC-CuvVD6yKd91prdKdohn5vGthce5-SrJG9hZQlFe_nuhi3z-Dl-07SJhtxggA2e23e1rs9-W04EYEMl5
- '@type': type.googleapis.com/google.rpc.PreconditionFailure
  violations:
  - subject: ?error_code=210002&type=Project&resource_id=durable-utility-367514
    type: googleapis.com
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: serviceusage.googleapis.com
  metadata:
    resource_id: durable-utility-367514
    type: Project
  reason: RESOURCES_NOT_FOUND


In [9]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ERROR: (gcloud.projects.get-iam-policy) User [selen.akkaya@modelway.it] does not have permission to access projects instance [durable-utility-367514:getIamPolicy] (or it may not exist): The caller does not have permission


In [10]:
!pip install kfp -U -q
!pip install google-cloud-pipeline-components -U -q

^C
^C


---
## Update AIPlatform Package:

The `google-cloud-aiplatform` package updates frequently.  Update it for latest functionality.

- [aiplatform Python Client](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
- [GitHub Repo for api-common-protos](https://github.com/googleapis/api-common-protos)

For a better understanding of the Vertex AI APIs client, version, and layers please review the tip here [aiplatform_notes.md](../Tips/aiplatform_notes.md).

In [11]:
!pip install googleapis-common-protos -U -q
!pip install google-cloud-aiplatform -U -q

^C
^C


# MODEL IMPLEMENTATION

In [12]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

gcs = storage.Client(project = PROJECT_ID)
BUCKET = PROJECT_ID

NameError: name 'storage' is not defined

In [13]:
# source data
RAIL_PROJECT = PROJECT_ID
RAIL_DATASET = 'rail-demo-367913'

In [14]:
import tensorflow as tf
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [15]:
"""

#to download files from cloud storage
bucket_name = "rail-demo-367913"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "saved_arrays_gray_scale/mesh/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

for blob in blobs:
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        df = np.load(file_name) # load the data
        print(df)
"""


'\n\n#to download files from cloud storage\nbucket_name = "rail-demo-367913"\n\nstorage_client = storage.Client()\nbucket = storage_client.get_bucket(bucket_name)\n\n# When you have your files in a subfolder of the bucket.\nmy_prefix = "saved_arrays_gray_scale/mesh/" # the name of the subfolder\nblobs = bucket.list_blobs(prefix = my_prefix, delimiter = \'/\')\n\nfor blob in blobs:\n    if(blob.name != my_prefix): # ignoring the subfolder itself \n        file_name = blob.name.replace(my_prefix, "")\n        blob.download_to_filename(file_name) # download the file to the machine\n        df = np.load(file_name) # load the data\n        print(df)\n'

In [16]:
#set parametrs
EPOCHS = 250
PATIENCE = 50
BATCH_SIZE = 8
OPTIMIZER = 'adam' #or rmsprop
DIM = 512
MODEL_NAME = 'mesh_unet_plus_mesh.h5'

CATEGORY = 'mesh'


In [17]:
import numpy as np
import random
import cv2
import os
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K
from evaluation_updated import mean_iou_test, dice_coeff, pixel_accuracy, pixel_accuracy_class1

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\hp\\anaconda3\\lib\\site-packages\\typing_extensions-3.10.0.2.dist-info\\'



ModuleNotFoundError: No module named 'cv2'

In [18]:
tf.random.set_seed(1)
src_arr = 'saved_arrays_gray_scale' 

NameError: name 'tf' is not defined

In [19]:
#load images and masks for train val and test. Normalize images in [0,1]. Expand dims of masks in the last channel
#choose if adding augmented images or not


train_images = np.load(src_arr + "/" + CATEGORY + "/gray_train_images.npy")
cw_images = np.load(src_arr + "/" + CATEGORY + "/gray_cw_images.npy") #images without mesh
train_images = np.concatenate((train_images, cw_images), axis=0)
train_images = train_images/255.0

train_masks = np.load(src_arr + "/" + CATEGORY + "/mesh_train_masks.npy")
cw_masks = np.load(src_arr + "/" + CATEGORY + "/cw_masks.npy")
train_masks = np.concatenate((train_masks, cw_masks), axis=0)
train_masks = np.expand_dims(train_masks, axis=-1)

val_images = np.load(src_arr + "/" + CATEGORY + "/gray_val_images.npy")
val_images = val_images/255.0
val_masks = np.load(src_arr + "/" + CATEGORY + "/mesh_val_masks.npy")
val_masks = np.expand_dims(val_masks, axis=-1)

test_images = np.load(src_arr + "/" + CATEGORY + "/gray_test_images.npy")
test_images = test_images/255.0
test_masks = np.load(src_arr + "/" + CATEGORY + "/mesh_test_masks.npy")
test_masks = np.expand_dims(test_masks, axis=-1)

print(train_images.shape)
print(train_masks.shape)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.


NameError: name 'src_arr' is not defined

In [20]:
sample_dir = "samples_mesh_gray_UNetPlus" #where to save the predictions
if not os.path.exists(sample_dir):
    os.mkdir(sample_dir)

checkpoint_path = "checkpoints_gray_UNetPlus" #where to save the model checkpoints
if not os.path.exists(checkpoint_path):
    os.mkdir(checkpoint_path)

NameError: name 'os' is not defined

In [21]:
def plot_img_and_masks(raw, mask):
    # image + prediction mask + target mask
    pred = model.predict(np.expand_dims(raw, 0))
    pred_msk = pred.squeeze()
    pred_msk = np.stack((pred_msk,) * 3, axis=-1)
    pred_msk[pred_msk >= 0.5] = 1
    pred_msk[pred_msk < 0.5] = 0
    target_msk = np.stack((mask,) * 3, axis=-1)
    raw = np.float32(raw)
    raw = cv2.cvtColor(raw, cv2.COLOR_RGB2BGR)
    combined = np.concatenate([raw, pred_msk, target_msk], axis=1)
    return combined

In [22]:

def build_callbacks():
    checkpointer = [
          EarlyStopping(monitor="val_mean_iou", patience=PATIENCE, restore_best_weights=True, mode="max"),
          ModelCheckpoint(checkpoint_path + '/' + MODEL_NAME, monitor="val_mean_iou", save_best_only=True, mode='max')
    ]
    return checkpointer

from unet_plus_model import model_build_func
model = model_build_func((DIM,DIM,1), 1, False)
model.summary()

ModuleNotFoundError: No module named 'unet_plus_model'

In [23]:

"""## Training"""

model.fit(train_images, train_masks, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(val_images, val_masks), callbacks = build_callbacks())


NameError: name 'model' is not defined